# Feature Engineering II: Column Transformers and Pipelines
<img src = "./images/lego.webp" width = "450">

<a href = "https://www.highsnobiety.com/p/lego-transformers-optimus-prime/">Image Source</a>

This notebook build on the [feature engineering introduction notebook](1.1_intro_to_fe.ipynb) to automate the transformation process, simplifying our workflow and unlocking the potential of the sklearn library.
<hr style="border:2px solid black">

## Penguin Dataset

We will use the Palmer Penguin Dataset.

### Business Goal
> Predict the penguin body mass given the input feature : flipper_length_mm, bill_length_mm, species and sex

#### Load Packages

In [1]:
# data analysis stack
import numpy as np
import pandas as pd

# data visualization stack
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style('whitegrid')

# machine-learning stack
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    KBinsDiscretizer,
    PolynomialFeatures
)

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# miscellaneous
import warnings
warnings.filterwarnings("ignore")

#### Load Data

In [2]:
df = pd.read_csv('./data/penguins.csv')
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
4,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male


#### Features and Target

In [3]:
numerical_features = [
    'flipper_length_mm',
    'bill_length_mm'
]

categorical_features = [
    'species',
    'sex'
]

features = numerical_features + categorical_features

target_variable = 'body_mass_g'

#### Feature-Target separation

In [4]:
# Feature matrix 
X = df[features]

# Target column
y = df[target_variable]

#### Train-Test Split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=88, shuffle=True, stratify=X['species'])

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (256, 4)
Test shape: (86, 4)


## Exploratory Data Analysis
Check issues with data:
+ which variable has missing values?
+ which variables are binary, categorical, metric?
+ do categorical variables have non-numeric values?
+ do metric features are varying on a different scale?


In [6]:
# Assuming X_train is a DataFrame and y_train is a Series
df_train = pd.concat([X_train, y_train], axis=1)

print("Combined train data shape:", df_train.shape)

Combined train data shape: (256, 5)


In [7]:
df_train.isna().sum()

flipper_length_mm    1
bill_length_mm       0
species              0
sex                  5
body_mass_g          0
dtype: int64

<hr style="border:2px solid black">

## Feature Engineering
We have a pair of tools, `ColumnTransformer()` and `Pipeline()`, which can dramatically simplify and automate feature engineering.


### ColumnTransformer()
<a href="https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html">`ColumnTransformer` </a> allows us to specify which columns receive which transformations (and conveniently reintegrates the dataset).

Parameters:
 * `transformers` - list of tuples `(name, transformer, columns)`

 * `remainder` - used as last tuple if there are any untouched columns. Choose either `drop` or `passthrough`<br></br>
  
>**Note** that `ColumnTransformer()` runs all transformers in parallel, not sequentially, so if a column is transformed more than once, the version generated by each of these transformations will be included.

#### Building our first transformer


In [8]:
# define our transformers - name, method, target
transformers = [('ohe', OneHotEncoder(drop = 'first',sparse_output=False), ['species', 'sex']),
                ('bill_scaler', RobustScaler(), [['bill_length_mm', 'flipper_length_mm']]),
                #('flip_scaler', RobustScaler(), ['flipper_length_mm'])
               ]

In [26]:
# now we instantiate our ColumnTransformer() object
column_transformer = ColumnTransformer(transformers,
                                       remainder = 'drop')
column_transformer

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('ohe', ...), ('bill_scaler', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``

We still need to impute missing values in sex and flipper_length_mm, but if we do so in this transformer we will create an imputed copy of sex and flipper_length_mm and a one-hot encoded versions with missing values.

What we need here is a way to sequentially apply transformations, which leads us nicely into...

### Pipeline()

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html">`Pipeline()`</a> allows us to sequentially apply multiple transformers on the same column(s).


Parameters:
 * `steps` - list of tuples `(name, transformer)`

#### Build a pipeline and integrate it into our transformer

In [10]:
# Let's define the steps to impute and transform sex
sex_steps = [('imputer', SimpleImputer(strategy = 'most_frequent')),
             ('sex_ohe', OneHotEncoder(drop = 'first',sparse_output=False))
             ]

In [12]:
# Let's define the steps to impute and transform flipper_length_mm
flipper_steps = [('imputer', SimpleImputer(strategy = 'median')),
             ('flipper_scaler', RobustScaler())
             ]

#### Let's Try It Out!

In [16]:
X_train.isna().sum()

flipper_length_mm    1
bill_length_mm       0
species              0
sex                  5
dtype: int64

In [27]:
# Transform the data
X_train_fe = column_transformer_2.transform(X_train)
X_train_fe

,sex_pipeline__sex_Male,ohe__species_Chinstrap,ohe__species_Gentoo,flipper_pipeline__flipper_length_mm,scaler__bill_length_mm
140,1.0,0.0,0.0,-0.430108,-0.440000
45,1.0,0.0,0.0,-0.645161,-0.382857
193,1.0,1.0,0.0,-0.043011,0.737143
286,1.0,0.0,1.0,1.376344,0.577143
37,0.0,0.0,0.0,-0.688172,-0.782857
...,...,...,...,...,...
245,0.0,0.0,1.0,0.817204,0.005714
31,0.0,0.0,0.0,-0.387097,-0.565714
82,1.0,0.0,0.0,-0.172043,-1.068571
56,1.0,0.0,0.0,-0.172043,-0.440000


As you can see the variables have been tranformed according to the strategies defined in our pipeline. 

## Model Building

### Nesting Pipelines

We've already seen how we can use essentially any named function or object as a step in our pipelines and transformers. The final trick we'll explore with pipelines is the ability to nest several layers within one.

In [18]:
# build a pipeline containing our complete transformer and then a linear regression model
model_steps = [('feature_enginnering', column_transformer_2),
               ('linear_regression', LinearRegression())]
linear_model = Pipeline(steps = model_steps)
linear_model

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('feature_enginnering', ...), ('linear_regression', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('sex_pipeline', ...), ('ohe', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output

**train model**

In [19]:
linear_model.fit(X_train,y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('feature_enginnering', ...), ('linear_regression', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('sex_pipeline', ...), ('ohe', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output

In [20]:
training_score = linear_model.score(X_train,y_train)
print(f"training r2 score: {round(training_score, 6)}")

training r2 score: 0.862912


### Model Evaluation

**Model Weigths**

In [21]:
column_step = linear_model.steps[0][1]
column_step

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('sex_pipeline', ...), ('ohe', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_

In [22]:
model_step = linear_model.steps[1][1]
model_step

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [23]:
coef_model = pd.DataFrame(data=model_step.coef_.reshape(1,-1), columns=column_step.get_feature_names_out(), index=['weigth'])

coef_model['intercept'] = model_step.intercept_
coef_model

,sex_pipeline__sex_Male,ohe__species_Chinstrap,ohe__species_Gentoo,flipper_pipeline__flipper_length_mm,scaler__bill_length_mm,intercept
weigth,421.527891,-360.368873,628.276557,406.147438,261.616625,3766.922464


**Model Prediction**

In [24]:
y_pred_test = linear_model.predict(X_test)
y_pred_test

array([4748.42170602, 3919.74689153, 4280.91244613, 4271.4746251 ,
       3864.18266383, 4480.41129253, 3426.76642453, 4688.1529015 ,
       4078.84551886, 4746.06811232, 4540.06984327, 3401.435034  ,
       3935.16452114, 5293.33757526, 3397.03297349, 4824.27634257,
       3879.30038771, 4087.34451298, 3065.76124063, 3321.48346383,
       3855.21556154, 3555.96861954, 5504.67934229, 3911.41348926,
       3532.18630969, 4130.14722108, 3271.12580966, 3334.55011032,
       5388.37820191, 3324.16563065, 5284.67298928, 4618.11248167,
       5225.34562224, 3320.54202635, 5417.33580732, 4189.47197755,
       3392.46271056, 4151.0765511 , 4044.38143418, 3404.73006518,
       5517.27527004, 4668.16500896, 5214.32744257, 3339.58848141,
       4015.58942063, 3465.3300534 , 3604.44339876, 3345.56828999,
       3284.18984557, 5559.13393008, 3499.16043805, 3438.88902296,
       3345.56828999, 4679.65390737, 3095.66289409, 4119.76274142,
       4342.12268813, 3371.53599111, 4170.12300617, 4014.01167

**Model Performance**

In [25]:
test_score = linear_model.score(X_test,y_test)
print(f"test r2 score: {round(test_score, 6)}")

test r2 score: 0.872736


>By applying a pipeline to our test data, we ensure that the test data is treated the in exact same way as the data the model was trained on. 

<hr style="border:2px solid black">